In [1]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../fuzzy-tools') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [2]:
%load_ext autoreload
%autoreload 2
from synthsne.results import synth_method_statistics as sms

kf = '1'
lcset_name = 'train'
cfilename = f'survey=alerceZTFv7.1~bands=gr~mode=onlySNe'
methods = [
    'linear-fstw',
    'bspline-fstw',
    'spm-mle-fstw',
    'spm-mle-estw',
    'spm-mcmc-fstw',
    'spm-mcmc-estw',
    ]

In [3]:
for method in methods:
    any_incorrects_fittings = sms.get_any_incorrects_fittings(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(f'method={method} - any_incorrects_fittings={len(any_incorrects_fittings)} #')
    #print(any_incorrects_fittings)
    
for method in methods:
    all_incorrects_fittings = sms.get_all_incorrects_fittings(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    #print(all_incorrects_fittings)
    print(f'method={method} - all_incorrects_fittings={len(all_incorrects_fittings)} #')

method=linear-fstw - any_incorrects_fittings=24 #
method=bspline-fstw - any_incorrects_fittings=24 #
method=spm-mle-fstw - any_incorrects_fittings=24 #
method=spm-mle-estw - any_incorrects_fittings=24 #
method=spm-mcmc-fstw - any_incorrects_fittings=24 #
method=spm-mcmc-estw - any_incorrects_fittings=24 #
method=linear-fstw - all_incorrects_fittings=0 #
method=bspline-fstw - all_incorrects_fittings=0 #
method=spm-mle-fstw - all_incorrects_fittings=0 #
method=spm-mle-estw - all_incorrects_fittings=0 #
method=spm-mcmc-fstw - all_incorrects_fittings=0 #
method=spm-mcmc-estw - all_incorrects_fittings=0 #


In [4]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms

for method in methods:
    xe_ptimes = sms.get_perf_times(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(f'method={method} - xe_ptimes={xe_ptimes}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'synth_method_statistics' from 'synthsne' (../synthsne/__init__.py)

In [ ]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms
from fuzzytools.latex.latex_tables import LatexTable

info_df = sms.get_info_dict(f'../save/ssne', methods, cfilename, kf, lcset_name)
latex_kwargs = {
    'caption':f'set={kf}@train',
    'label':'???',
    'custom_tabular_align':'lc|'+'c'*sum([m.split('-')[-1]=='fstw' for m in methods])+'|'+'c'*sum([m.split('-')[-1]=='estw' for m in methods]), 
}
latex_table = LatexTable(info_df, **latex_kwargs)
print(latex_table)
info_df

In [ ]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms

for method in methods:
    rank_bdict = sms.get_ranks(f'../save/ssne/{method}/{cfilename}', kf, lcset_name)
    print(method)
    for b in rank_bdict.keys():
        print(rank_bdict[b])

In [ ]:
%load_ext autoreload
%autoreload 2
from synthsne import synth_method_statistics as sms
from fuzzytools.files import load_pickle
from fuzzytools.dataframes import DFBuilder
from fuzzytools.latex.latex_tables import LatexTable

kf = '0'
lcset_name = 'train'
cfilename = f'survey=alerceZTFv7.1~bands=gr~mode=onlySNe'
filedir = f'../save/mcmc_priors/{cfilename}/{kf}@{lcset_name}/mcmc_priors.d'
priors_d = load_pickle(filedir)
info_df = DFBuilder()
spms = ['A','t0','gamma','f','trise','tfall']
for b in ['g', 'r']:
    for spm_p in spms:
        d = {}
        for c in ['SLSN', 'SNIa', 'SNIbc', 'allSNII']:
            d[c] = priors_d[b][c][spm_p]
    
        info_df.append(f'SPM var={spm_p}~band={b}', d)
    
info_df = info_df.get_df()
latex_kwargs = {
    'caption':f'set={kf}@train',
    'label':'???',
    'hline_k':len(spms)-1,
    }
latex_table = LatexTable(info_df, **latex_kwargs)
print(latex_table)
info_df